### HR Data

In [66]:
import pandas as pd
import re
from collections import defaultdict
import json

# Load input data from a .json file
with open('/home/lohith/Downloads/tao_digital/HRDATA.json', 'r') as file:
    input_data = json.load(file)

# Create a defaultdict to store data for each TS_id
ts_data = defaultdict(dict)

# Iterate over the input data and organize it by TS_id
for key, value in input_data.items():
    if key != "timestamp":
        match = re.match(r"TS\[(\d+)\]\.(\w+)\.(\w+)", key)
        if match:
            ts_id, category, attribute = match.groups()
            ts_data[ts_id][f"{category}.{attribute}"] = value

# Convert the defaultdict to a list of dictionaries
list_of_dicts = [{"timestamp": input_data["timestamp"], "TS_id": ts_id, **data} for ts_id, data in ts_data.items()]

# Create a DataFrame from the list of dictionaries
df = pd.DataFrame(list_of_dicts)

df['site_id'] = 'relavant_path1'

# Reorder columns
columns_order = [
     'timestamp', 'TS_id', 'site_id',
    'system.SystemState', 'system.BMS_EMS_CommStatus', 'system.SafetyLoopStatus',
    'system.SystemStatusFlags', 'system.CapacityChargekW', 'system.CapacityDischargekW',
    'system.CapacitykVAR', 'system.SysVoltageDC', 'system.SysCurrentDC',
    'bms.BMSState', 'bms.EventLog_LastEventNum',
    'pcs.PCS_MaxCapacitykW', 'pcs.PCS_MaxCapacitykVAR', 'pcs.PCSStatus_All_PCS',
    'pcs.PF_Command_All_PCS', 'pcs.Actual_PF_All_PCS', 'pcs.P_Command_All_PCS',
    'pcs.P_CommandAck_All_PCS', 'pcs.Q_Command_All_PCS', 'pcs.Q_CommandAck_All_PCS',
    'pcs.RealPower_All_PCS', 'pcs.ReactivePower_All_PCS', 'pcs.ApparentPower_All_PCS'
]

df = df[columns_order]

# Display the processed DataFrame
df

,timestamp,TS_id,site_id,system.SystemState,system.BMS_EMS_CommStatus,system.SafetyLoopStatus,system.SystemStatusFlags,system.CapacityChargekW,system.CapacityDischargekW,system.CapacitykVAR,...,pcs.PCSStatus_All_PCS,pcs.PF_Command_All_PCS,pcs.Actual_PF_All_PCS,pcs.P_Command_All_PCS,pcs.P_CommandAck_All_PCS,pcs.Q_Command_All_PCS,pcs.Q_CommandAck_All_PCS,pcs.RealPower_All_PCS,pcs.ReactivePower_All_PCS,pcs.ApparentPower_All_PCS
0,2023-10-19 11:00:05.6 PDT,1,relavant_path1,4,0,0,3423076352,3600,-3600,2520,...,1073741824,0,0,0,450,0,0,450,0,450
1,2023-10-19 11:00:05.6 PDT,2,relavant_path1,4,0,0,3423076352,3600,-3600,2520,...,1073741824,0,0,0,450,0,0,450,0,450
2,2023-10-19 11:00:05.6 PDT,3,relavant_path1,4,0,0,3423076352,3600,-3600,2520,...,1073741824,0,0,0,450,0,0,450,0,450
3,2023-10-19 11:00:05.6 PDT,4,relavant_path1,4,0,0,3423076352,3600,-3600,2520,...,1073741824,0,0,0,450,0,0,450,0,450
4,2023-10-19 11:00:05.6 PDT,5,relavant_path1,4,0,0,3423076352,3600,-3600,2520,...,1073741824,0,0,0,450,0,0,450,0,450
5,2023-10-19 11:00:05.6 PDT,6,relavant_path1,4,0,0,3423076352,3600,-3600,2520,...,1073741824,0,0,0,450,0,0,450,0,450
6,2023-10-19 11:00:05.6 PDT,7,relavant_path1,4,0,0,3423076352,3600,-3600,2520,...,1073741824,0,0,0,450,0,0,450,0,450
7,2023-10-19 11:00:05.6 PDT,8,relavant_path1,4,0,0,3423076352,3600,-3600,2520,...,1073741824,0,0,0,450,0,0,450,0,450


In [63]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col, to_timestamp

# Initialize Spark session
spark = SparkSession.builder.appName("TelemetryProcessing").getOrCreate()

# JSON input data
json_file_path = "/home/lohith/Downloads/relative_path1/hrdata_schema.json"

# Load JSON data into a DataFrame
df = spark.read.json(json_file_path)

# Convert timestamp to the proper format
df = df.withColumn("timestamp", to_timestamp(col("timestamp"), "yyyy-MM-dd HH:mm:ss.S z"))

# Define a function to extract information from the nested structure
def extract_info(ts_id, data):
    return (
        ts_id,
        data["timestamp"],
        data[f"TS[{ts_id}].system.SystemState"],
        data[f"TS[{ts_id}].system.BMS_EMS_CommStatus"],
        data[f"TS[{ts_id}].system.SafetyLoopStatus"],
        data[f"TS[{ts_id}].system.SystemStatusFlags"],
        data[f"TS[{ts_id}].system.CapacityChargekW"],
        data[f"TS[{ts_id}].system.CapacityDischargekW"],
        data[f"TS[{ts_id}].system.CapacitykVAR"],
        data[f"TS[{ts_id}].system.SysVoltageDC"],
        data[f"TS[{ts_id}].system.SysCurrentDC"],
        data[f"TS[{ts_id}].bms.BMSState"],
        data[f"TS[{ts_id}].bms.EventLog_LastEventNum"],
        data[f"TS[{ts_id}].pcs.PCS_MaxCapacitykW"],
        data[f"TS[{ts_id}].pcs.PCS_MaxCapacitykVAR"],
        data[f"TS[{ts_id}].pcs.PCSStatus_All_PCS"],
        data[f"TS[{ts_id}].pcs.PF_Command_All_PCS"],
        data[f"TS[{ts_id}].pcs.Actual_PF_All_PCS"],
        data[f"TS[{ts_id}].pcs.P_Command_All_PCS"],
        data[f"TS[{ts_id}].pcs.P_CommandAck_All_PCS"],
        data[f"TS[{ts_id}].pcs.Q_Command_All_PCS"],
        data[f"TS[{ts_id}].pcs.Q_CommandAck_All_PCS"],
        data[f"TS[{ts_id}].pcs.RealPower_All_PCS"],
        data[f"TS[{ts_id}].pcs.ReactivePower_All_PCS"],
        data[f"TS[{ts_id}].pcs.ApparentPower_All_PCS"]
    )

# Extract information for each TS and union the results
ts_ids = range(1, 9)
processed_data = spark.sparkContext.parallelize(
    [extract_info(ts_id, df.first()) for ts_id in ts_ids]
)

result_df = spark.createDataFrame(processed_data, [
    "TS_id",
    "timestamp",
    "system.SystemState",
    "system.BMS_EMS_CommStatus",
    "system.SafetyLoopStatus",
    "system.SystemStatusFlags",
    "system.CapacityChargekW",
    "system.CapacityDischargekW",
    "system.CapacitykVAR",
    "system.SysVoltageDC",
    "system.SysCurrentDC",
    "bms.BMSState",
    "bms.EventLog_LastEventNum",
    "pcs.PCS_MaxCapacitykW",
    "pcs.PCS_MaxCapacitykVAR",
    "pcs.PCSStatus_All_PCS",
    "pcs.PF_Command_All_PCS",
    "pcs.Actual_PF_All_PCS",
    "pcs.P_Command_All_PCS",
    "pcs.P_CommandAck_All_PCS",
    "pcs.Q_Command_All_PCS",
    "pcs.Q_CommandAck_All_PCS",
    "pcs.RealPower_All_PCS",
    "pcs.ReactivePower_All_PCS",
    "pcs.ApparentPower_All_PCS"
])

# Write the result to a CSV file
result_df.coalesce(1).write.option("header", "true").csv("output_file1.csv")

# Stop the Spark session
spark.stop()

AnalysisException: [UNRESOLVED_COLUMN.WITH_SUGGESTION] A column or function parameter with name `timestamp` cannot be resolved. Did you mean one of the following? [`Comment`, `Name`, `Type`, `_corrupt_record`].;
'Project [Comment#371, Name#372, Type#373, _corrupt_record#374, to_timestamp('timestamp, Some(yyyy-MM-dd HH:mm:ss.S z), TimestampType, Some(Asia/Kolkata), false) AS timestamp#379]
+- Relation [Comment#371,Name#372,Type#373,_corrupt_record#374] json


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, LongType, DoubleType

# Your existing Spark session initialization
spark = SparkSession.builder.appName("TelemetryProcessing").getOrCreate()

# Specify the path to your JSON file
json_file_path = "/home/lohith/Downloads/relative_path1/hrdata_schema.json"

# Load JSON data from a file into a DataFrame with the specified schema
df = spark.read.schema(schema).json(json_file_path)

# Load JSON data into a DataFrame with the specified schema
df = spark.read.schema(schema).json(spark.sparkContext.parallelize([json_data]))

# Your existing Spark session initialization
spark = SparkSession.builder.appName("TelemetryProcessing").getOrCreate()

# Define the schema based on the JSON structure
schema = StructType([
    StructField("TS_id", IntegerType()),
    StructField("timestamp", StringType()),
    StructField("system.SystemState", IntegerType()),
    StructField("system.BMS_EMS_CommStatus", IntegerType()),
    StructField("system.SafetyLoopStatus", IntegerType()),
    StructField("system.SystemStatusFlags", LongType()),
    StructField("system.CapacityChargekW", DoubleType()),
    StructField("system.CapacityDischargekW", DoubleType()),
    StructField("system.CapacitykVAR", DoubleType()),
    StructField("system.SysVoltageDC", IntegerType()),
    StructField("system.SysCurrentDC", IntegerType()),
    StructField("bms.BMSState", IntegerType()),
    StructField("bms.EventLog_LastEventNum", IntegerType()),
    StructField("pcs.PCS_MaxCapacitykW", DoubleType()),
    StructField("pcs.PCS_MaxCapacitykVAR", DoubleType()),
    StructField("pcs.PCSStatus_All_PCS", LongType()),
    StructField("pcs.PF_Command_All_PCS", IntegerType()),
    StructField("pcs.Actual_PF_All_PCS", IntegerType()),
    StructField("pcs.P_Command_All_PCS", IntegerType()),
    StructField("pcs.P_CommandAck_All_PCS", IntegerType()),
    StructField("pcs.Q_Command_All_PCS", IntegerType()),
    StructField("pcs.Q_CommandAck_All_PCS", IntegerType()),
    StructField("pcs.RealPower_All_PCS", DoubleType()),
    StructField("pcs.ReactivePower_All_PCS", DoubleType()),
    StructField("pcs.ApparentPower_All_PCS", DoubleType())
])


# Define a function to extract information from the nested structure
def extract_info(row):
    return (
        row["TS_id"],
        row["timestamp"],
        row["system.SystemState"],
        row["system.BMS_EMS_CommStatus"],
        row["system.SafetyLoopStatus"],
        row["system.SystemStatusFlags"],
        row["system.CapacityChargekW"],
        row["system.CapacityDischargekW"],
        row["system.CapacitykVAR"],
        row["system.SysVoltageDC"],
        row["system.SysCurrentDC"],
        row["bms.BMSState"],
        row["bms.EventLog_LastEventNum"],
        row["pcs.PCS_MaxCapacitykW"],
        row["pcs.PCS_MaxCapacitykVAR"],
        row["pcs.PCSStatus_All_PCS"],
        row["pcs.PF_Command_All_PCS"],
        row["pcs.Actual_PF_All_PCS"],
        row["pcs.P_Command_All_PCS"],
        row["pcs.P_CommandAck_All_PCS"],
        row["pcs.Q_Command_All_PCS"],
        row["pcs.Q_CommandAck_All_PCS"],
        row["pcs.RealPower_All_PCS"],
        row["pcs.ReactivePower_All_PCS"],
        row["pcs.ApparentPower_All_PCS"]
    )

# Process data in parallel
processed_data = df.rdd.map(lambda row: extract_info(row)).collect()

# Create DataFrame from the processed data
result_df = spark.createDataFrame(processed_data, schema)

# Write the result to a CSV file
result_df.write.option("header", "true").csv("output_file1.csv")

# Stop the Spark session
spark.stop()

# Write the result to a CSV file
result_df.write.option("header", "true").csv("output_file1.csv")

# Stop the Spark session
spark.stop()

### HR_data

In [ ]:
# Assuming schema is defined explicitly based on the JSON structure
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, ...

# Assuming schema is defined explicitly based on the JSON structure
schema = StructType([
    StructField("TS_id", IntegerType()),
    StructField("timestamp", StringType()),
    # ... (define other fields with appropriate data types)
])

df = spark.read.schema(schema).json(spark.sparkContext.parallelize([json_data]))



# Create DataFrame from the processed data
result_df = spark.createDataFrame(processed_data, schema)

# Write the result to a CSV file
result_df.write.option("header", "true").csv("output_file1.csv")


In [ ]:
pwd

In [ ]:
result_df

In [ ]:
df.show(truncate=False)

In [ ]:
from pyspark.sql.functions import col

# Assuming your DataFrame is named df
pf_command_ts7 = df.select(col("TS[7].pcs.PF_Command_All_PCS")).collect()

# This will give you a list of Row objects, you can access the value using the first element
value_pf_command_ts7 = pf_command_ts7[0]["TS[7].pcs.PF_Command_All_PCS"]

# Print the value
print(value_pf_command_ts7)

In [ ]:
df.printSchema()

In [ ]:
# Assuming your DataFrame is named df
pf_command_ts7 = df.select(col("TS[7].pcs.PF_Command_All_PCS")).collect()
value_pf_command_ts7 = pf_command_ts7[0]["TS[7].pcs.PF_Command_All_PCS"]
print(value_pf_command_ts7)

In [ ]:
result_df.show()

In [ ]:
from pyspark.sql import SparkSession
import json
import os

# Initialize Spark session
spark = SparkSession.builder.appName("TelemetryProcessing").getOrCreate()

# Define the path to the JSON file
json_file_path = '/home/lohith/Downloads/relative_path1/hrdata_schema.json'

# Extract the site_id from the directory name
site_id = os.path.basename(os.path.dirname(json_file_path))

# Read JSON data from the file
with open(json_file_path, 'r') as file:
    json_data = json.load(file)

# Create an empty list to store processed data
processed_data = []

# Print keys in the JSON data
print(json_data.keys())

# Iterate through the TS data in the JSON and extract information
for ts_key, ts_data in json_data.items():
    if ts_key.startswith("TS[") and ts_key.endswith("]"):
        ts_id = int(ts_key.split("[")[1].split("]")[0])
        processed_data.append((
            site_id,
            json_data["timestamp"],
            ts_id,
            ts_data["system"]["SystemState"],
            ts_data["system"]["BMS_EMS_CommStatus"],
            ts_data["system"]["SafetyLoopStatus"],
            ts_data["system"]["SystemStatusFlags"],
            ts_data["system"]["CapacityChargekW"],
            ts_data["system"]["CapacityDischargekW"],
            ts_data["system"]["CapacitykVAR"],
            ts_data["system"]["SysVoltageDC"],
            ts_data["system"]["SysCurrentDC"],
            ts_data["bms"]["BMSState"],
            ts_data["bms"]["EventLog_LastEventNum"],
            ts_data["pcs"]["PCS_MaxCapacitykW"],
            ts_data["pcs"]["PCS_MaxCapacitykVAR"],
            ts_data["pcs"]["PCSStatus_All_PCS"],
            ts_data["pcs"]["PF_Command_All_PCS"],
            ts_data["pcs"]["Actual_PF_All_PCS"],
            ts_data["pcs"]["P_Command_All_PCS"],
            ts_data["pcs"]["P_CommandAck_All_PCS"],
            ts_data["pcs"]["Q_Command_All_PCS"],
            ts_data["pcs"]["Q_CommandAck_All_PCS"],
            ts_data["pcs"]["RealPower_All_PCS"],
            ts_data["pcs"]["ReactivePower_All_PCS"],
            ts_data["pcs"]["ApparentPower_All_PCS"]
        ))


# Define column names
columns = [
    "site_id",
    "timestamp",
    "TS_id",
    "system.SystemState",
    "system.BMS_EMS_CommStatus",
    "system.SafetyLoopStatus",
    "system.SystemStatusFlags",
    "system.CapacityChargekW",
    "system.CapacityDischargekW",
    "system.CapacitykVAR",
    "system.SysVoltageDC",
    "system.SysCurrentDC",
    "bms.BMSState",
    "bms.EventLog_LastEventNum",
    "pcs.PCS_MaxCapacitykW",
    "pcs.PCS_MaxCapacitykVAR",
    "pcs.PCSStatus_All_PCS",
    "pcs.PF_Command_All_PCS",
    "pcs.Actual_PF_All_PCS",
    "pcs.P_Command_All_PCS",
    "pcs.P_CommandAck_All_PCS",
    "pcs.Q_Command_All_PCS",
    "pcs.Q_CommandAck_All_PCS",
    "pcs.RealPower_All_PCS",
    "pcs.ReactivePower_All_PCS",
    "pcs.ApparentPower_All_PCS"
]

# Create a DataFrame from the processed data
result_df = spark.createDataFrame(processed_data, columns)

# Write the result to a CSV file
result_df.coalesce(1).write.option("header", "true").csv(f"output_{site_id}.csv")

# Stop the Spark session
spark.stop()


In [ ]:
df.show()

In [ ]:
print(json_data)

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Initialize Spark session
spark = SparkSession.builder.appName("TelemetryProcessing").getOrCreate()

# Define the path to the JSON file
json_file_path = '/home/lohith/Downloads/relative_path1/hrdata_schema.json'

# Extract the site_id from the directory name
site_id = os.path.basename(os.path.dirname(json_file_path))

# Read JSON data from the file
with open(json_file_path, 'r') as file:
    json_data = file.read()

# Load JSON data into a DataFrame
df = spark.read.json(spark.sparkContext.parallelize([json_data]))

# Identify the timestamp column dynamically
timestamp_column = "timestamp"  # Assuming 'timestamp' is the key in your JSON file

# Extract the TS_id from the column names
ts_id_column = col("timestamp").substr(4, 1).alias("TS_id")

# Define a list of expressions for selectExpr
select_expr = [
    ts_id_column,
    col("timestamp")
]

# Add expressions for each column dynamically
for col_name in df.columns:
    if col_name != "timestamp":
        ts_id = col_name.split(".")[0].split("[")[1]  # Extract TS_id from the column name
        expr = col(col_name).alias(f"{col_name.split(']')[1][1:]}_{ts_id}")
        select_expr.append(expr)

# Apply selectExpr to the DataFrame
result_df = df.selectExpr(select_expr)

# Write the result to a CSV file
result_df.coalesce(1).write.option("header", "true").csv(f"output_{site_id}.csv")

# Stop the Spark session
spark.stop()


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Initialize Spark session
spark = SparkSession.builder.appName("TelemetryProcessing").getOrCreate()

# JSON input data
json_data = '/home/lohith/Downloads/relative_path1/hrdata_schema.json'

# Load JSON data into a DataFrame
df = spark.read.json(spark.sparkContext.parallelize([json_data]))

# Define a function to extract information from the nested structure
def extract_info(ts_id, data):
    return (
        ts_id,
        data["timestamp"],
        data[f"TS[{ts_id}].system.SystemState"],
        data[f"TS[{ts_id}].system.BMS_EMS_CommStatus"],
        data[f"TS[{ts_id}].system.SafetyLoopStatus"],
        data[f"TS[{ts_id}].system.SystemStatusFlags"],
        data[f"TS[{ts_id}].system.CapacityChargekW"],
        data[f"TS[{ts_id}].system.CapacityDischargekW"],
        data[f"TS[{ts_id}].system.CapacitykVAR"],
        data[f"TS[{ts_id}].system.SysVoltageDC"],
        data[f"TS[{ts_id}].system.SysCurrentDC"],
        data[f"TS[{ts_id}].bms.BMSState"],
        data[f"TS[{ts_id}].bms.EventLog_LastEventNum"],
        data[f"TS[{ts_id}].pcs.PCS_MaxCapacitykW"],
        data[f"TS[{ts_id}].pcs.PCS_MaxCapacitykVAR"],
        data[f"TS[{ts_id}].pcs.PCSStatus_All_PCS"],
        data[f"TS[{ts_id}].pcs.PF_Command_All_PCS"],
        data[f"TS[{ts_id}].pcs.Actual_PF_All_PCS"],
        data[f"TS[{ts_id}].pcs.P_Command_All_PCS"],
        data[f"TS[{ts_id}].pcs.P_CommandAck_All_PCS"],
        data[f"TS[{ts_id}].pcs.Q_Command_All_PCS"],
        data[f"TS[{ts_id}].pcs.Q_CommandAck_All_PCS"],
        data[f"TS[{ts_id}].pcs.RealPower_All_PCS"],
        data[f"TS[{ts_id}].pcs.ReactivePower_All_PCS"],
        data[f"TS[{ts_id}].pcs.ApparentPower_All_PCS"]
    )

# Extract information for each TS and union the results
ts_ids = range(1, 9)
processed_data = []
for ts_id in ts_ids:
    processed_data.append(extract_info(ts_id, df.first()))

result_df = spark.createDataFrame(processed_data, [
    "TS_id",
    "timestamp",
    "system.SystemState",
    "system.BMS_EMS_CommStatus",
    "system.SafetyLoopStatus",
    "system.SystemStatusFlags",
    "system.CapacityChargekW",
    "system.CapacityDischargekW",
    "system.CapacitykVAR",
    "system.SysVoltageDC",
    "system.SysCurrentDC",
    "bms.BMSState",
    "bms.EventLog_LastEventNum",
    "pcs.PCS_MaxCapacitykW",
    "pcs.PCS_MaxCapacitykVAR",
    "pcs.PCSStatus_All_PCS",
    "pcs.PF_Command_All_PCS",
    "pcs.Actual_PF_All_PCS",
    "pcs.P_Command_All_PCS",
    "pcs.P_CommandAck_All_PCS",
    "pcs.Q_Command_All_PCS",
    "pcs.Q_CommandAck_All_PCS",
    "pcs.RealPower_All_PCS",
    "pcs.ReactivePower_All_PCS",
    "pcs.ApparentPower_All_PCS"
])

# Show the DataFrame
result_df.show(truncate=False)

# Write the result to a CSV file
# result_df.coalesce(1).write.option("header", "true").csv("output_file1.csv")

# Stop the Spark session
spark.stop()


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import lit
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, FloatType

# Initialize Spark session
spark = SparkSession.builder.appName("TelemetryProcessing").getOrCreate()

# JSON input data
json_data = '/home/lohith/Downloads/relative_path1/hrdata_schema.json'

# Load JSON data into a DataFrame
df = spark.read.json(spark.sparkContext.parallelize([json_data]))

# Define a function to extract information from the nested structure
def extract_info(ts_id, timestamp, data):
    return (
        ts_id,
        timestamp,
        data[f"TS[{ts_id}].system.SystemState"],
        data[f"TS[{ts_id}].system.BMS_EMS_CommStatus"],
        data[f"TS[{ts_id}].system.SafetyLoopStatus"],
        data[f"TS[{ts_id}].system.SystemStatusFlags"],
        data[f"TS[{ts_id}].system.CapacityChargekW"],
        data[f"TS[{ts_id}].system.CapacityDischargekW"],
        data[f"TS[{ts_id}].system.CapacitykVAR"],
        data[f"TS[{ts_id}].system.SysVoltageDC"],
        data[f"TS[{ts_id}].system.SysCurrentDC"],
        data[f"TS[{ts_id}].bms.BMSState"],
        data[f"TS[{ts_id}].bms.EventLog_LastEventNum"],
        data[f"TS[{ts_id}].pcs.PCS_MaxCapacitykW"],
        data[f"TS[{ts_id}].pcs.PCS_MaxCapacitykVAR"],
        data[f"TS[{ts_id}].pcs.PCSStatus_All_PCS"],
        data[f"TS[{ts_id}].pcs.PF_Command_All_PCS"],
        data[f"TS[{ts_id}].pcs.Actual_PF_All_PCS"],
        data[f"TS[{ts_id}].pcs.P_Command_All_PCS"],
        data[f"TS[{ts_id}].pcs.P_CommandAck_All_PCS"],
        data[f"TS[{ts_id}].pcs.Q_Command_All_PCS"],
        data[f"TS[{ts_id}].pcs.Q_CommandAck_All_PCS"],
        data[f"TS[{ts_id}].pcs.RealPower_All_PCS"],
        data[f"TS[{ts_id}].pcs.ReactivePower_All_PCS"],
        data[f"TS[{ts_id}].pcs.ApparentPower_All_PCS"]
    )

# Define the constant timestamp value
constant_timestamp = "2023-10-19 11:00:05.6 PDT"

# Extract information for each TS and union the results
ts_ids = range(1, 9)
processed_data = []
for ts_id in ts_ids:
    processed_data.append(extract_info(ts_id, constant_timestamp, df.first()))

# Define the schema for the DataFrame
schema = StructType([
    StructField("TS_id", IntegerType(), True),
    StructField("timestamp", StringType(), True),
    StructField("system_SystemState", IntegerType(), True),
    StructField("system_BMS_EMS_CommStatus", IntegerType(), True),
    StructField("system_SafetyLoopStatus", IntegerType(), True),
    StructField("system_SystemStatusFlags", IntegerType(), True),
    StructField("system_CapacityChargekW", IntegerType(), True),
    StructField("system_CapacityDischargekW", IntegerType(), True),
    StructField("system_CapacitykVAR", IntegerType(), True),
    StructField("system_SysVoltageDC", IntegerType(), True),
    StructField("system_SysCurrentDC", IntegerType(), True),
    StructField("bms_BMSState", IntegerType(), True),
    StructField("bms_EventLog_LastEventNum", IntegerType(), True),
    StructField("pcs_PCS_MaxCapacitykW", IntegerType(), True),
    StructField("pcs_PCS_MaxCapacitykVAR", IntegerType(), True),
    StructField("pcs_PCSStatus_All_PCS", IntegerType(), True),
    StructField("pcs_PF_Command_All_PCS", IntegerType(), True),
    StructField("pcs_Actual_PF_All_PCS", IntegerType(), True),
    StructField("pcs_P_Command_All_PCS", IntegerType(), True),
    StructField("pcs_P_CommandAck_All_PCS", IntegerType(), True),
    StructField("pcs_Q_Command_All_PCS", IntegerType(), True),
    StructField("pcs_Q_CommandAck_All_PCS", IntegerType(), True),
    StructField("pcs_RealPower_All_PCS", IntegerType(), True),
    StructField("pcs_ReactivePower_All_PCS", IntegerType(), True),
    StructField("pcs_ApparentPower_All_PCS", IntegerType(), True)
])

# Create DataFrame
result_df = spark.createDataFrame(processed_data, schema)

# Show the DataFrame
result_df.show(truncate=False)

# Write the result to a CSV file
# result_df.coalesce(1).write.option("header", "true").csv("output_file1.csv")

# Stop the Spark session
spark.stop()


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, StringType
import re

# Initialize Spark session
spark = SparkSession.builder.appName("TelemetryProcessing").getOrCreate()

# JSON input data
json_data = '/home/lohith/Downloads/relative_path1/hrdata_schema.json'

# Load JSON data into a DataFrame
df = spark.read.json(spark.sparkContext.parallelize([json_data]))

# Define a function to extract information from the nested structure
def extract_info(ts_id, data):
    def replace_special_chars(col_name):
        # Replace special characters with underscores
        return col_name.replace(".", "_").replace("[", "_").replace("]", "_")

    return (
        ts_id,
        "2023-10-19 11:00:05.6 PDT",  # Fixed timestamp value
        data[replace_special_chars(f"TS[{ts_id}].system.SystemState")],
        data[replace_special_chars(f"TS[{ts_id}].system.BMS_EMS_CommStatus")],
        data[replace_special_chars(f"TS[{ts_id}].system.SafetyLoopStatus")],
        data[replace_special_chars(f"TS[{ts_id}].system.SystemStatusFlags")],
        data[replace_special_chars(f"TS[{ts_id}].system.CapacityChargekW")],
        data[replace_special_chars(f"TS[{ts_id}].system.CapacityDischargekW")],
        data[replace_special_chars(f"TS[{ts_id}].system.CapacitykVAR")],
        data[replace_special_chars(f"TS[{ts_id}].system.SysVoltageDC")],
        data[replace_special_chars(f"TS[{ts_id}].system.SysCurrentDC")],
        data[replace_special_chars(f"TS[{ts_id}].bms.BMSState")],
        data[replace_special_chars(f"TS[{ts_id}].bms.EventLog_LastEventNum")],
        data[replace_special_chars(f"TS[{ts_id}].pcs.PCS_MaxCapacitykW")],
        data[replace_special_chars(f"TS[{ts_id}].pcs.PCS_MaxCapacitykVAR")],
        data[replace_special_chars(f"TS[{ts_id}].pcs.PCSStatus_All_PCS")],
        data[replace_special_chars(f"TS[{ts_id}].pcs.PF_Command_All_PCS")],
        data[replace_special_chars(f"TS[{ts_id}].pcs.Actual_PF_All_PCS")],
        data[replace_special_chars(f"TS[{ts_id}].pcs.P_Command_All_PCS")],
        data[replace_special_chars(f"TS[{ts_id}].pcs.P_CommandAck_All_PCS")],
        data[replace_special_chars(f"TS[{ts_id}].pcs.Q_Command_All_PCS")],
        data[replace_special_chars(f"TS[{ts_id}].pcs.Q_CommandAck_All_PCS")],
        data[replace_special_chars(f"TS[{ts_id}].pcs.RealPower_All_PCS")],
        data[replace_special_chars(f"TS[{ts_id}].pcs.ReactivePower_All_PCS")],
        data[replace_special_chars(f"TS[{ts_id}].pcs.ApparentPower_All_PCS")]
    )

# Extract information for each TS and union the results
ts_ids = range(1, 9)
processed_data = []
for ts_id in ts_ids:
    processed_data.append(extract_info(ts_id, df.first()))

# Define the schema for the DataFrame
schema = StructType([
    StructField("TS_id", StringType(), True),
    StructField("timestamp", StringType(), True),
    StructField("system_SystemState", StringType(), True),
    StructField("system_BMS_EMS_CommStatus", StringType(), True),
    StructField("system_SafetyLoopStatus", StringType(), True),
    StructField("system_SystemStatusFlags", StringType(), True),
    StructField("system_CapacityChargekW", StringType(), True),
    StructField("system_CapacityDischargekW", StringType(), True),
    StructField("system_CapacitykVAR", StringType(), True),
    StructField("system_SysVoltageDC", StringType(), True),
    StructField("system_SysCurrentDC", StringType(), True),
    StructField("bms_BMSState", StringType(), True),
    StructField("bms_EventLog_LastEventNum", StringType(), True),
    StructField("pcs_PCS_MaxCapacitykW", StringType(), True),
    StructField("pcs_PCS_MaxCapacitykVAR", StringType(), True),
    StructField("pcs_PCSStatus_All_PCS", StringType(), True),
    StructField("pcs_PF_Command_All_PCS", StringType(), True),
    StructField("pcs_Actual_PF_All_PCS", StringType(), True),
    StructField("pcs_P_Command_All_PCS", StringType(), True),
    StructField("pcs_P_CommandAck_All_PCS", StringType(), True),
    StructField("pcs_Q_Command_All_PCS", StringType(), True),
    StructField("pcs_Q_CommandAck_All_PCS", StringType(), True),
    StructField("pcs_RealPower_All_PCS", StringType(), True),
    StructField("pcs_ReactivePower_All_PCS", StringType(), True),
    StructField("pcs_ApparentPower_All_PCS", StringType(), True),
])

# Replace special characters in column names with underscores in the schema
for field in schema.fields:
    field.name = re.sub(r'[^a-zA-Z0-9_]', '_', field.name)

# Create DataFrame
result_df = spark.createDataFrame(processed_data, schema)

# Show the DataFrame
result_df.show(truncate=False)

# Write the result to a CSV file
# result_df.coalesce(1).write.csv('/path/to/output/folder', header=True, mode='overwrite')


In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Initialize Spark session
spark = SparkSession.builder.appName("TelemetryProcessing").getOrCreate()

# JSON input data
json_data = '/home/lohith/Downloads/relative_path1/hrdata_schema.json'

# Load JSON data into a DataFrame
df = spark.read.json(spark.sparkContext.parallelize([json_data]))

# Define a function to extract information from the nested structure
def extract_info(ts_id, data):
    return (
        ts_id,
        "2023-10-19 11:00:05.6 PDT",
        "relative_path1",
        data[f"TS[{ts_id}].system.SystemState"],
        data[f"TS[{ts_id}].system.BMS_EMS_CommStatus"],
        data[f"TS[{ts_id}].system.SafetyLoopStatus"],
        data[f"TS[{ts_id}].system.SystemStatusFlags"],
        data[f"TS[{ts_id}].system.CapacityChargekW"],
        data[f"TS[{ts_id}].system.CapacityDischargekW"],
        data[f"TS[{ts_id}].system.CapacitykVAR"],
        data[f"TS[{ts_id}].system.SysVoltageDC"],
        data[f"TS[{ts_id}].system.SysCurrentDC"],
        data[f"TS[{ts_id}].bms.BMSState"],
        data[f"TS[{ts_id}].bms.EventLog_LastEventNum"],
        data[f"TS[{ts_id}].pcs.PCS_MaxCapacitykW"],
        data[f"TS[{ts_id}].pcs.PCS_MaxCapacitykVAR"],
        data[f"TS[{ts_id}].pcs.PCSStatus_All_PCS"],
        data[f"TS[{ts_id}].pcs.PF_Command_All_PCS"],
        data[f"TS[{ts_id}].pcs.Actual_PF_All_PCS"],
        data[f"TS[{ts_id}].pcs.P_Command_All_PCS"],
        data[f"TS[{ts_id}].pcs.P_CommandAck_All_PCS"],
        data[f"TS[{ts_id}].pcs.Q_Command_All_PCS"],
        data[f"TS[{ts_id}].pcs.Q_CommandAck_All_PCS"],
        data[f"TS[{ts_id}].pcs.RealPower_All_PCS"],
        data[f"TS[{ts_id}].pcs.ReactivePower_All_PCS"],
        data[f"TS[{ts_id}].pcs.ApparentPower_All_PCS"]
    )

# Extract information for each TS and union the results
ts_ids = range(1, 9)
processed_data = []
for ts_id in ts_ids:
    processed_data.append(extract_info(ts_id, df.first()))

result_df = spark.createDataFrame(processed_data, [
    "TS_id",
    "timestamp",
    "site_id",
    "system.SystemState",
    "system.BMS_EMS_CommStatus",
    "system.SafetyLoopStatus",
    "system.SystemStatusFlags",
    "system.CapacityChargekW",
    "system.CapacityDischargekW",
    "system.CapacitykVAR",
    "system.SysVoltageDC",
    "system.SysCurrentDC",
    "bms.BMSState",
    "bms.EventLog_LastEventNum",
    "pcs.PCS_MaxCapacitykW",
    "pcs.PCS_MaxCapacitykVAR",
    "pcs.PCSStatus_All_PCS",
    "pcs.PF_Command_All_PCS",
    "pcs.Actual_PF_All_PCS",
    "pcs.P_Command_All_PCS",
    "pcs.P_CommandAck_All_PCS",
    "pcs.Q_Command_All_PCS",
    "pcs.Q_CommandAck_All_PCS",
    "pcs.RealPower_All_PCS",
    "pcs.ReactivePower_All_PCS",
    "pcs.ApparentPower_All_PCS"
])

# Write the result to a CSV file
result_df.coalesce(1).write.option("header", "true").csv("output_file5.csv")

# Stop the Spark session
spark.stop()

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col

# Initialize Spark session
spark = SparkSession.builder.appName("TelemetryProcessing").getOrCreate()

# JSON input data
json_data = '/home/lohith/Downloads/relative_path1/hrdata_schema.json'

# Load JSON data into a DataFrame
df = spark.read.json(json_data)

# Define a function to extract information from the nested structure
def extract_info(ts_id, data):
    return (
        ts_id,
        "2023-10-19 11:00:05.6 PDT",
        "relative_path1",
        data[ts_id]["system"]["SystemState"],
        data[ts_id]["system"]["BMS_EMS_CommStatus"],
        data[ts_id]["system"]["SafetyLoopStatus"],
        data[ts_id]["system"]["SystemStatusFlags"],
        data[ts_id]["system"]["CapacityChargekW"],
        data[ts_id]["system"]["CapacityDischargekW"],
        data[ts_id]["system"]["CapacitykVAR"],
        data[ts_id]["system"]["SysVoltageDC"],
        data[ts_id]["system"]["SysCurrentDC"],
        data[ts_id]["bms"]["BMSState"],
        data[ts_id]["bms"]["EventLog_LastEventNum"],
        data[ts_id]["pcs"]["PCS_MaxCapacitykW"],
        data[ts_id]["pcs"]["PCS_MaxCapacitykVAR"],
        data[ts_id]["pcs"]["PCSStatus_All_PCS"],
        data[ts_id]["pcs"]["PF_Command_All_PCS"],
        data[ts_id]["pcs"]["Actual_PF_All_PCS"],
        data[ts_id]["pcs"]["P_Command_All_PCS"],
        data[ts_id]["pcs"]["P_CommandAck_All_PCS"],
        data[ts_id]["pcs"]["Q_Command_All_PCS"],
        data[ts_id]["pcs"]["Q_CommandAck_All_PCS"],
        data[ts_id]["pcs"]["RealPower_All_PCS"],
        data[ts_id]["pcs"]["ReactivePower_All_PCS"],
        data[ts_id]["pcs"]["ApparentPower_All_PCS"]
    )

# Extract information for each TS and union the results
ts_ids = [f"TS[{i}]" for i in range(1, 9)]
processed_data = []
for ts_id in ts_ids:
    processed_data.append(extract_info(ts_id, df.first()))

# Create DataFrame
result_df = spark.createDataFrame(processed_data, [
    "TS_id",
    "timestamp",
    "site_id",
    "system_SystemState",
    "system_BMS_EMS_CommStatus",
    "system_SafetyLoopStatus",
    "system_SystemStatusFlags",
    "system_CapacityChargekW",
    "system_CapacityDischargekW",
    "system_CapacitykVAR",
    "system_SysVoltageDC",
    "system_SysCurrentDC",
    "bms_BMSState",
    "bms_EventLog_LastEventNum",
    "pcs_PCS_MaxCapacitykW",
    "pcs_PCS_MaxCapacitykVAR",
    "pcs_PCSStatus_All_PCS",
    "pcs_PF_Command_All_PCS",
    "pcs_Actual_PF_All_PCS",
    "pcs_P_Command_All_PCS",
    "pcs_P_CommandAck_All_PCS",
    "pcs_Q_Command_All_PCS",
    "pcs_Q_CommandAck_All_PCS",
    "pcs_RealPower_All_PCS",
    "pcs_ReactivePower_All_PCS",
    "pcs_ApparentPower_All_PCS"
])

# Write the result to a CSV file
result_df.coalesce(1).write.option("header", "true").csv("output_file5.csv")

# Stop the Spark session
spark.stop()


In [7]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

# Initialize Spark session
spark = SparkSession.builder.appName("TelemetryProcessing").getOrCreate()

# JSON input data
json_data = '/home/lohith/Downloads/relative_path1/hrdata_schema.json'

# Load JSON data into a DataFrame
df = spark.read.json(json_data)

# Define the schema
schema = StructType([
    StructField("timestamp", StringType(), True),
    StructField("TS[1].system.SystemState", IntegerType(), True),
    StructField("TS[1].system.BMS_EMS_CommStatus", IntegerType(), True),
    StructField("TS[1].system.SafetyLoopStatus", IntegerType(), True),
    StructField("TS[1].system.SystemStatusFlags", IntegerType(), True),
    StructField("TS[1].system.CapacityChargekW", IntegerType(), True),
    StructField("TS[1].system.CapacityDischargekW", IntegerType(), True),
    StructField("TS[1].system.CapacitykVAR", IntegerType(), True),
    StructField("TS[1].system.SysVoltageDC", IntegerType(), True),
    StructField("TS[1].system.SysCurrentDC", IntegerType(), True),
    StructField("TS[1].bms.BMSState", IntegerType(), True),
    StructField("TS[1].bms.EventLog_LastEventNum", IntegerType(), True),
    StructField("TS[1].pcs.PCS_MaxCapacitykW", IntegerType(), True),
    StructField("TS[1].pcs.PCS_MaxCapacitykVAR", IntegerType(), True),
    StructField("TS[1].pcs.PCSStatus_All_PCS", IntegerType(), True),
    StructField("TS[1].pcs.PF_Command_All_PCS", IntegerType(), True),
    StructField("TS[1].pcs.Actual_PF_All_PCS", IntegerType(), True),
    StructField("TS[1].pcs.P_Command_All_PCS", IntegerType(), True),
    StructField("TS[1].pcs.P_CommandAck_All_PCS", IntegerType(), True),
    StructField("TS[1].pcs.Q_Command_All_PCS", IntegerType(), True),
    StructField("TS[1].pcs.Q_CommandAck_All_PCS", IntegerType(), True),
    StructField("TS[1].pcs.RealPower_All_PCS", IntegerType(), True),
    StructField("TS[1].pcs.ReactivePower_All_PCS", IntegerType(), True),
    StructField("TS[1].pcs.ApparentPower_All_PCS", IntegerType(), True),
    # Add similar StructField entries for other TS ids
])

# Load JSON data with the specified schema
df = spark.read.schema(schema).json(json_data)

# Define a function to extract information from the nested structure
def extract_info(ts_id, data):
    return (
        ts_id,
        "2023-10-19 11:00:05.6 PDT",
        "relative_path1",
        data.get(f"TS[{ts_id}].system.SystemState", None),
        data.get(f"TS[{ts_id}].system.BMS_EMS_CommStatus", None),
        data.get(f"TS[{ts_id}].system.SafetyLoopStatus", None),
        data.get(f"TS[{ts_id}].system.SystemStatusFlags", None),
        data.get(f"TS[{ts_id}].system.CapacityChargekW", None),
        data.get(f"TS[{ts_id}].system.CapacityDischargekW", None),
        data.get(f"TS[{ts_id}].system.CapacitykVAR", None),
        data.get(f"TS[{ts_id}].system.SysVoltageDC", None),
        data.get(f"TS[{ts_id}].system.SysCurrentDC", None),
        data.get(f"TS[{ts_id}].bms.BMSState", None),
        data.get(f"TS[{ts_id}].bms.EventLog_LastEventNum", None),
        data.get(f"TS[{ts_id}].pcs.PCS_MaxCapacitykW", None),
        data.get(f"TS[{ts_id}].pcs.PCS_MaxCapacitykVAR", None),
        data.get(f"TS[{ts_id}].pcs.PCSStatus_All_PCS", None),
        data.get(f"TS[{ts_id}].pcs.PF_Command_All_PCS", None),
        data.get(f"TS[{ts_id}].pcs.Actual_PF_All_PCS", None),
        data.get(f"TS[{ts_id}].pcs.P_Command_All_PCS", None),
        data.get(f"TS[{ts_id}].pcs.P_CommandAck_All_PCS", None),
        data.get(f"TS[{ts_id}].pcs.Q_Command_All_PCS", None),
        data.get(f"TS[{ts_id}].pcs.Q_CommandAck_All_PCS", None),
        data.get(f"TS[{ts_id}].pcs.RealPower_All_PCS", None),
        data.get(f"TS[{ts_id}].pcs.ReactivePower_All_PCS", None),
        data.get(f"TS[{ts_id}].pcs.ApparentPower_All_PCS", None)
    )




# Extract information for each TS and union the results
ts_ids = range(1, 9)
processed_data = []
for row in df.collect():
    for ts_id in ts_ids:
        processed_data.append(extract_info(ts_id, row))

result_df = spark.createDataFrame(processed_data, [
    "TS_id",
    "timestamp",
    "site_id",
    "system_SystemState",
    "system_BMS_EMS_CommStatus",
    "system_SafetyLoopStatus",
    "system_SystemStatusFlags",
    "system_CapacityChargekW",
    "system_CapacityDischargekW",
    "system_CapacitykVAR",
    "system_SysVoltageDC",
    "system_SysCurrentDC",
    "bms_BMSState",
    "bms_EventLog_LastEventNum",
    "pcs_PCS_MaxCapacitykW",
    "pcs_PCS_MaxCapacitykVAR",
    "pcs_PCSStatus_All_PCS",
    "pcs_PF_Command_All_PCS",
    "pcs_Actual_PF_All_PCS",
    "pcs_P_Command_All_PCS",
    "pcs_P_CommandAck_All_PCS",
    "pcs_Q_Command_All_PCS",
    "pcs_Q_CommandAck_All_PCS",
    "pcs_RealPower_All_PCS",
    "pcs_ReactivePower_All_PCS",
    "pcs_ApparentPower_All_PCS"
])

# Write the result to a CSV file
result_df.repartition(1).write.option("header", "true").csv("output_file5.csv")

# Stop the Spark session
spark.stop()


AttributeError: get

In [6]:
# Test if the 'get' method works in your Python environment
test_dict = {"key": "value"}
test_result = test_dict.get("key", None)
print(test_result)

value


In [8]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import col
from pyspark.sql.types import StructType, StructField, StringType, IntegerType

# Initialize Spark session
spark = SparkSession.builder.appName("TelemetryProcessing").getOrCreate()

# JSON input data
json_data = '/home/lohith/Downloads/relative_path1/hrdata_schema.json'

# Load JSON data into a DataFrame
df = spark.read.json(json_data)

# Define the schema
schema = StructType([
    StructField("timestamp", StringType(), True),
    StructField("TS[1].system.SystemState", IntegerType(), True),
    StructField("TS[1].system.BMS_EMS_CommStatus", IntegerType(), True),
    StructField("TS[1].system.SafetyLoopStatus", IntegerType(), True),
    StructField("TS[1].system.SystemStatusFlags", IntegerType(), True),
    StructField("TS[1].system.CapacityChargekW", IntegerType(), True),
    StructField("TS[1].system.CapacityDischargekW", IntegerType(), True),
    StructField("TS[1].system.CapacitykVAR", IntegerType(), True),
    StructField("TS[1].system.SysVoltageDC", IntegerType(), True),
    StructField("TS[1].system.SysCurrentDC", IntegerType(), True),
    StructField("TS[1].bms.BMSState", IntegerType(), True),
    StructField("TS[1].bms.EventLog_LastEventNum", IntegerType(), True),
    StructField("TS[1].pcs.PCS_MaxCapacitykW", IntegerType(), True),
    StructField("TS[1].pcs.PCS_MaxCapacitykVAR", IntegerType(), True),
    StructField("TS[1].pcs.PCSStatus_All_PCS", IntegerType(), True),
    StructField("TS[1].pcs.PF_Command_All_PCS", IntegerType(), True),
    StructField("TS[1].pcs.Actual_PF_All_PCS", IntegerType(), True),
    StructField("TS[1].pcs.P_Command_All_PCS", IntegerType(), True),
    StructField("TS[1].pcs.P_CommandAck_All_PCS", IntegerType(), True),
    StructField("TS[1].pcs.Q_Command_All_PCS", IntegerType(), True),
    StructField("TS[1].pcs.Q_CommandAck_All_PCS", IntegerType(), True),
    StructField("TS[1].pcs.RealPower_All_PCS", IntegerType(), True),
    StructField("TS[1].pcs.ReactivePower_All_PCS", IntegerType(), True),
    StructField("TS[1].pcs.ApparentPower_All_PCS", IntegerType(), True),
    # Add similar StructField entries for other TS ids
])

# Load JSON data with the specified schema
df = spark.read.schema(schema).json(json_data)


# Define a function to extract information from the nested structure
def extract_info(ts_id, data):
    return (
        ts_id,
        "2023-10-19 11:00:05.6 PDT",
        "relative_path1",
        data[f"TS[{ts_id}].system.SystemState"],
        data[f"TS[{ts_id}].system.BMS_EMS_CommStatus"],
        data[f"TS[{ts_id}].system.SafetyLoopStatus"],
        data[f"TS[{ts_id}].system.SystemStatusFlags"],
        data[f"TS[{ts_id}].system.CapacityChargekW"],
        data[f"TS[{ts_id}].system.CapacityDischargekW"],
        data[f"TS[{ts_id}].system.CapacitykVAR"],
        data[f"TS[{ts_id}].system.SysVoltageDC"],
        data[f"TS[{ts_id}].system.SysCurrentDC"],
        data[f"TS[{ts_id}].bms.BMSState"],
        data[f"TS[{ts_id}].bms.EventLog_LastEventNum"],
        data[f"TS[{ts_id}].pcs.PCS_MaxCapacitykW"],
        data[f"TS[{ts_id}].pcs.PCS_MaxCapacitykVAR"],
        data[f"TS[{ts_id}].pcs.PCSStatus_All_PCS"],
        data[f"TS[{ts_id}].pcs.PF_Command_All_PCS"],
        data[f"TS[{ts_id}].pcs.Actual_PF_All_PCS"],
        data[f"TS[{ts_id}].pcs.P_Command_All_PCS"],
        data[f"TS[{ts_id}].pcs.P_CommandAck_All_PCS"],
        data[f"TS[{ts_id}].pcs.Q_Command_All_PCS"],
        data[f"TS[{ts_id}].pcs.Q_CommandAck_All_PCS"],
        data[f"TS[{ts_id}].pcs.RealPower_All_PCS"],
        data[f"TS[{ts_id}].pcs.ReactivePower_All_PCS"],
        data[f"TS[{ts_id}].pcs.ApparentPower_All_PCS"]
    )


# Extract information for each TS and union the results
ts_ids = range(1, 9)
processed_data = []
for ts_id in ts_ids:
    processed_data.append(extract_info(ts_id, df.first().asDict()))

result_df = spark.createDataFrame(processed_data, [
    "TS_id",
    "timestamp",
    "site_id",
    "system.SystemState",
    "system.BMS_EMS_CommStatus",
    # Add similar column names for other columns
])

# Write the result to a CSV file
result_df.coalesce(1).write.option("header", "true").csv("output_file5.csv")

# Stop the Spark session
spark.stop()

In [13]:
df['system.SystemStatusFlags']

0    0
1    0
2    0
3    0
4    0
5    0
6    0
7    0
Name: system.SystemStatusFlags, dtype: int64

In [19]:
import pandas as pd
import re

# Load your input data
input_data = {
    "timestamp": "2023-10-19 11:00:05.6 PDT",
    "TS[1].system.SystemState": 4,
    # ... (all other entries)
    "TS[8].pcs.ReactivePower_All_PCS": 0
}

# Function to extract information from the nested structure
def extract_info(data):
    # Initialize a list to store extracted values
    extracted_data_list = []
    
    # Extract data for each TS entry
    for key, value in data.items():
        if key != "timestamp":
            match = re.match(r"TS\[(\d+)\]\.(\w+)\.(\w+)", key)
            if match:
                ts_id, category, attribute = match.groups()
                extracted_data = {
                    "timestamp": data.get("timestamp", ""),
                    "TS_id": int(ts_id),
                    "site_id": category,
                    f"{category}.{attribute}": value
                }
                extracted_data_list.append(extracted_data)

    return extracted_data_list

# Extract data from the input
extracted_data_list = extract_info(input_data)

# Create a DataFrame from the extracted data list
df = pd.DataFrame(extracted_data_list)

# Display the resulting DataFrame
print(df)



                   timestamp  TS_id site_id  system.SystemState  \
0  2023-10-19 11:00:05.6 PDT      1  system                 4.0   
1  2023-10-19 11:00:05.6 PDT      8     pcs                 NaN   

   pcs.ReactivePower_All_PCS  
0                        NaN  
1                        0.0  


In [21]:
import pandas as pd
import re

# Load your input data
input_data = {
    "timestamp": "2023-10-19 11:00:05.6 PDT",
    "TS[1].system.SystemState": 4,
    # ... (all other entries)
    "TS[8].pcs.ReactivePower_All_PCS": 0
}

# Function to extract information from the nested structure
def extract_info(data):
    # Initialize a list to store extracted values
    extracted_data_list = []
    
    # Extract data for each TS entry
    for key, value in data.items():
        if key != "timestamp":
            match = re.match(r"TS\[(\d+)\]\.(\w+)\.(\w+)", key)
            if match:
                ts_id, category, attribute = match.groups()
                extracted_data = {
                    "timestamp": data.get("timestamp", ""),
                    "TS_id": int(ts_id),
                    "site_id": "relative_path1",
                    f"{category}.{attribute}": value
                }
                extracted_data_list.append(extracted_data)

    return extracted_data_list

# Extract data from the input
extracted_data_list = extract_info(input_data)

# Create a DataFrame from the extracted data list
df = pd.DataFrame(extracted_data_list)

# Display the resulting DataFrame
print(df)


                   timestamp  TS_id         site_id  system.SystemState  \
0  2023-10-19 11:00:05.6 PDT      1  relative_path1                 4.0   
1  2023-10-19 11:00:05.6 PDT      8  relative_path1                 NaN   

   pcs.ReactivePower_All_PCS  
0                        NaN  
1                        0.0  


In [35]:
import pandas as pd
import re

# Your input data
input_data = {
	"timestamp":	"2023-10-19 11:00:05.6 PDT",
	"TS[1].system.SystemState":	4,
	"TS[1].system.BMS_EMS_CommStatus":	0,
	"TS[1].system.SafetyLoopStatus":	0,
	"TS[1].system.SystemStatusFlags":	3423076352,
	"TS[1].system.CapacityChargekW":	3600,
	"TS[1].system.CapacityDischargekW":	-3600,
	"TS[1].system.CapacitykVAR":	2520,
	"TS[1].system.SysVoltageDC":	1453,
	"TS[1].system.SysCurrentDC":	32,
	"TS[1].bms.BMSState":	0,
	"TS[1].bms.EventLog_LastEventNum":	0,
	"TS[1].pcs.PCS_MaxCapacitykW":	3600,
	"TS[1].pcs.PCS_MaxCapacitykVAR":	2520,
	"TS[1].pcs.PCSStatus_All_PCS":	1073741824,
	"TS[1].pcs.PF_Command_All_PCS":	0,
	"TS[1].pcs.Actual_PF_All_PCS":	0,
	"TS[1].pcs.P_Command_All_PCS":	0,
	"TS[1].pcs.P_CommandAck_All_PCS":	450,
	"TS[1].pcs.Q_Command_All_PCS":	0,
	"TS[1].pcs.Q_CommandAck_All_PCS":	0,
	"TS[1].pcs.RealPower_All_PCS":	450,
	"TS[1].pcs.ReactivePower_All_PCS":	0,
	"TS[1].pcs.ApparentPower_All_PCS":	450,
	"TS[2].system.SystemState":	4,
	"TS[2].system.BMS_EMS_CommStatus":	0,
	"TS[2].system.SafetyLoopStatus":	0,
	"TS[2].system.SystemStatusFlags":	3423076352,
	"TS[2].system.CapacityChargekW":	3600,
	"TS[2].system.CapacityDischargekW":	-3600,
	"TS[2].system.CapacitykVAR":	2520,
	"TS[2].system.SysVoltageDC":	1453,
	"TS[2].system.SysCurrentDC":	32,
	"TS[2].bms.BMSState":	0,
	"TS[2].bms.EventLog_LastEventNum":	0,
	"TS[2].pcs.PCS_MaxCapacitykW":	3600,
	"TS[2].pcs.PCS_MaxCapacitykVAR":	2520,
	"TS[2].pcs.PCSStatus_All_PCS":	1073741824,
	"TS[2].pcs.PF_Command_All_PCS":	0,
	"TS[2].pcs.Actual_PF_All_PCS":	0,
	"TS[2].pcs.P_Command_All_PCS":	0,
	"TS[2].pcs.P_CommandAck_All_PCS":	450,
	"TS[2].pcs.Q_Command_All_PCS":	0,
	"TS[2].pcs.Q_CommandAck_All_PCS":	0,
	"TS[2].pcs.RealPower_All_PCS":	450,
	"TS[2].pcs.ReactivePower_All_PCS":	0,
	"TS[2].pcs.ApparentPower_All_PCS":	450,
	"TS[3].system.SystemState":	4,
	"TS[3].system.BMS_EMS_CommStatus":	0,
	"TS[3].system.SafetyLoopStatus":	0,
	"TS[3].system.SystemStatusFlags":	3423076352,
	"TS[3].system.CapacityChargekW":	3600,
	"TS[3].system.CapacityDischargekW":	-3600,
	"TS[3].system.CapacitykVAR":	2520,
	"TS[3].system.SysVoltageDC":	1453,
	"TS[3].system.SysCurrentDC":	32,
	"TS[3].bms.BMSState":	0,
	"TS[3].bms.EventLog_LastEventNum":	0,
	"TS[3].pcs.PCS_MaxCapacitykW":	3600,
	"TS[3].pcs.PCS_MaxCapacitykVAR":	2520,
	"TS[3].pcs.PCSStatus_All_PCS":	1073741824,
	"TS[3].pcs.PF_Command_All_PCS":	0,
	"TS[3].pcs.Actual_PF_All_PCS":	0,
	"TS[3].pcs.P_Command_All_PCS":	0,
	"TS[3].pcs.P_CommandAck_All_PCS":	450,
	"TS[3].pcs.Q_Command_All_PCS":	0,
	"TS[3].pcs.Q_CommandAck_All_PCS":	0,
	"TS[3].pcs.RealPower_All_PCS":	450,
	"TS[3].pcs.ReactivePower_All_PCS":	0,
	"TS[3].pcs.ApparentPower_All_PCS":	450,
	"TS[4].system.SystemState":	4,
	"TS[4].system.BMS_EMS_CommStatus":	0,
	"TS[4].system.SafetyLoopStatus":	0,
	"TS[4].system.SystemStatusFlags":	3423076352,
	"TS[4].system.CapacityChargekW":	3600,
	"TS[4].system.CapacityDischargekW":	-3600,
	"TS[4].system.CapacitykVAR":	2520,
	"TS[4].system.SysVoltageDC":	1453,
	"TS[4].system.SysCurrentDC":	32,
	"TS[4].bms.BMSState":	0,
	"TS[4].bms.EventLog_LastEventNum":	0,
	"TS[4].pcs.PCS_MaxCapacitykW":	3600,
	"TS[4].pcs.PCS_MaxCapacitykVAR":	2520,
	"TS[4].pcs.PCSStatus_All_PCS":	1073741824,
	"TS[4].pcs.PF_Command_All_PCS":	0,
	"TS[4].pcs.Actual_PF_All_PCS":	0,
	"TS[4].pcs.P_Command_All_PCS":	0,
	"TS[4].pcs.P_CommandAck_All_PCS":	450,
	"TS[4].pcs.Q_Command_All_PCS":	0,
	"TS[4].pcs.Q_CommandAck_All_PCS":	0,
	"TS[4].pcs.RealPower_All_PCS":	450,
	"TS[4].pcs.ReactivePower_All_PCS":	0,
	"TS[4].pcs.ApparentPower_All_PCS":	450,
	"TS[5].system.SystemState":	4,
	"TS[5].system.BMS_EMS_CommStatus":	0,
	"TS[5].system.SafetyLoopStatus":	0,
	"TS[5].system.SystemStatusFlags":	3423076352,
	"TS[5].system.CapacityChargekW":	3600,
	"TS[5].system.CapacityDischargekW":	-3600,
	"TS[5].system.CapacitykVAR":	2520,
	"TS[5].system.SysVoltageDC":	1453,
	"TS[5].system.SysCurrentDC":	32,
	"TS[5].bms.BMSState":	0,
	"TS[5].bms.EventLog_LastEventNum":	0,
	"TS[5].pcs.PCS_MaxCapacitykW":	3600,
	"TS[5].pcs.PCS_MaxCapacitykVAR":	2520,
	"TS[5].pcs.PCSStatus_All_PCS":	1073741824,
	"TS[5].pcs.PF_Command_All_PCS":	0,
	"TS[5].pcs.Actual_PF_All_PCS":	0,
	"TS[5].pcs.P_Command_All_PCS":	0,
	"TS[5].pcs.P_CommandAck_All_PCS":	450,
	"TS[5].pcs.Q_Command_All_PCS":	0,
	"TS[5].pcs.Q_CommandAck_All_PCS":	0,
	"TS[5].pcs.RealPower_All_PCS":	450,
	"TS[5].pcs.ReactivePower_All_PCS":	0,
	"TS[5].pcs.ApparentPower_All_PCS":	450,
	"TS[6].system.SystemState":	4,
	"TS[6].system.BMS_EMS_CommStatus":	0,
	"TS[6].system.SafetyLoopStatus":	0,
	"TS[6].system.SystemStatusFlags":	3423076352,
	"TS[6].system.CapacityChargekW":	3600,
	"TS[6].system.CapacityDischargekW":	-3600,
	"TS[6].system.CapacitykVAR":	2520,
	"TS[6].system.SysVoltageDC":	1453,
	"TS[6].system.SysCurrentDC":	32,
	"TS[6].bms.BMSState":	0,
	"TS[6].bms.EventLog_LastEventNum":	0,
	"TS[6].pcs.PCS_MaxCapacitykW":	3600,
	"TS[6].pcs.PCS_MaxCapacitykVAR":	2520,
	"TS[6].pcs.PCSStatus_All_PCS":	1073741824,
	"TS[6].pcs.PF_Command_All_PCS":	0,
	"TS[6].pcs.Actual_PF_All_PCS":	0,
	"TS[6].pcs.P_Command_All_PCS":	0,
	"TS[6].pcs.P_CommandAck_All_PCS":	450,
	"TS[6].pcs.Q_Command_All_PCS":	0,
	"TS[6].pcs.Q_CommandAck_All_PCS":	0,
	"TS[6].pcs.RealPower_All_PCS":	450,
	"TS[6].pcs.ReactivePower_All_PCS":	0,
	"TS[6].pcs.ApparentPower_All_PCS":	450,
	"TS[7].system.SystemState":	4,
	"TS[7].system.BMS_EMS_CommStatus":	0,
	"TS[7].system.SafetyLoopStatus":	0,
	"TS[7].system.SystemStatusFlags":	3423076352,
	"TS[7].system.CapacityChargekW":	3600,
	"TS[7].system.CapacityDischargekW":	-3600,
	"TS[7].system.CapacitykVAR":	2520,
	"TS[7].system.SysVoltageDC":	1453,
	"TS[7].system.SysCurrentDC":	32,
	"TS[7].bms.BMSState":	0,
	"TS[7].bms.EventLog_LastEventNum":	0,
	"TS[7].pcs.PCS_MaxCapacitykW":	3600,
	"TS[7].pcs.PCS_MaxCapacitykVAR":	2520,
	"TS[7].pcs.PCSStatus_All_PCS":	1073741824,
	"TS[7].pcs.PF_Command_All_PCS":	0,
	"TS[7].pcs.Actual_PF_All_PCS":	0,
	"TS[7].pcs.P_Command_All_PCS":	0,
	"TS[7].pcs.P_CommandAck_All_PCS":	450,
	"TS[7].pcs.Q_Command_All_PCS":	0,
	"TS[7].pcs.Q_CommandAck_All_PCS":	0,
	"TS[7].pcs.RealPower_All_PCS":	450,
	"TS[7].pcs.ReactivePower_All_PCS":	0,
	"TS[7].pcs.ApparentPower_All_PCS":	450,
	"TS[8].system.SystemState":	4,
	"TS[8].system.BMS_EMS_CommStatus":	0,
	"TS[8].system.SafetyLoopStatus":	0,
	"TS[8].system.SystemStatusFlags":	3423076352,
	"TS[8].system.CapacityChargekW":	3600,
	"TS[8].system.CapacityDischargekW":	-3600,
	"TS[8].system.CapacitykVAR":	2520,
	"TS[8].system.SysVoltageDC":	1453,
	"TS[8].system.SysCurrentDC":	32,
	"TS[8].bms.BMSState":	0,
	"TS[8].bms.EventLog_LastEventNum":	0,
	"TS[8].pcs.PCS_MaxCapacitykW":	3600,
	"TS[8].pcs.PCS_MaxCapacitykVAR":	2520,
	"TS[8].pcs.PCSStatus_All_PCS":	1073741824,
	"TS[8].pcs.PF_Command_All_PCS":	0,
	"TS[8].pcs.Actual_PF_All_PCS":	0,
	"TS[8].pcs.P_Command_All_PCS":	0,
	"TS[8].pcs.P_CommandAck_All_PCS":	450,
	"TS[8].pcs.Q_Command_All_PCS":	0,
	"TS[8].pcs.Q_CommandAck_All_PCS":	0,
	"TS[8].pcs.RealPower_All_PCS":	450,
	"TS[8].pcs.ReactivePower_All_PCS":	0,
	"TS[8].pcs.ApparentPower_All_PCS":	450
}

# Function to extract information from the nested structure
def extract_info(data):
    # Initialize a list to store extracted values
    extracted_data_list = []
    
    # Extract data for each TS entry
    for key, value in data.items():
        if key == "timestamp":
            timestamp = value
        else:
            match = re.match(r"TS\[(\d+)\]\.(\w+)\.(\w+)", key)
            if match:
                ts_id, category, attribute = match.groups()
                extracted_data = {
                    "timestamp": timestamp,
                    "TS_id": int(ts_id),
                    "site_id": "relative_path1",
                    f"{category}.{attribute}": value
                }
                extracted_data_list.append(extracted_data)

    return extracted_data_list

# Extract data from the input
extracted_data_list = extract_info(input_data)

# Create a DataFrame from the extracted data list
df = pd.DataFrame(extracted_data_list)

# Display the resulting DataFrame
print(df.to_csv(index=False))


timestamp,TS_id,site_id,system.SystemState,system.BMS_EMS_CommStatus,system.SafetyLoopStatus,system.SystemStatusFlags,system.CapacityChargekW,system.CapacityDischargekW,system.CapacitykVAR,system.SysVoltageDC,system.SysCurrentDC,bms.BMSState,bms.EventLog_LastEventNum,pcs.PCS_MaxCapacitykW,pcs.PCS_MaxCapacitykVAR,pcs.PCSStatus_All_PCS,pcs.PF_Command_All_PCS,pcs.Actual_PF_All_PCS,pcs.P_Command_All_PCS,pcs.P_CommandAck_All_PCS,pcs.Q_Command_All_PCS,pcs.Q_CommandAck_All_PCS,pcs.RealPower_All_PCS,pcs.ReactivePower_All_PCS,pcs.ApparentPower_All_PCS
2023-10-19 11:00:05.6 PDT,1,relative_path1,4.0,,,,,,,,,,,,,,,,,,,,,,
2023-10-19 11:00:05.6 PDT,1,relative_path1,,0.0,,,,,,,,,,,,,,,,,,,,,
2023-10-19 11:00:05.6 PDT,1,relative_path1,,,0.0,,,,,,,,,,,,,,,,,,,,
2023-10-19 11:00:05.6 PDT,1,relative_path1,,,,3423076352.0,,,,,,,,,,,,,,,,,,,
2023-10-19 11:00:05.6 PDT,1,relative_path1,,,,,3600.0,,,,,,,,,,,,,,,,,,
2023-10-19 11:00:05.6 PDT,1,relative_path1,,,,,,-3600.0,,,,,,,,,,,,,,,,,
2023-10-19 11:00:05.6

In [36]:
df

,timestamp,TS_id,site_id,system.SystemState,system.BMS_EMS_CommStatus,system.SafetyLoopStatus,system.SystemStatusFlags,system.CapacityChargekW,system.CapacityDischargekW,system.CapacitykVAR,...,pcs.PCSStatus_All_PCS,pcs.PF_Command_All_PCS,pcs.Actual_PF_All_PCS,pcs.P_Command_All_PCS,pcs.P_CommandAck_All_PCS,pcs.Q_Command_All_PCS,pcs.Q_CommandAck_All_PCS,pcs.RealPower_All_PCS,pcs.ReactivePower_All_PCS,pcs.ApparentPower_All_PCS
0,2023-10-19 11:00:05.6 PDT,1,relative_path1,4.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2023-10-19 11:00:05.6 PDT,1,relative_path1,NaN,0.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2023-10-19 11:00:05.6 PDT,1,relative_path1,NaN,NaN,0.0,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,2023-10-19 11:00:05.6 PDT,1,relative_path1,NaN,NaN,NaN,3.423076e+09,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,2023-10-19 11:00:05.6 PDT,1,relative_path1,NaN,NaN,NaN,NaN,3600.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
179,2023-10-19 11:00:05.6 PDT,8,relative_path1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN,NaN
180,2023-10-19 11:00:05.6 PDT,8,relative_path1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,NaN
181,2023-10-19 11:00:05.6 PDT,8,relative_path1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,450.0,NaN,NaN
182,2023-10-19 11:00:05.6 PDT,8,relative_path1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN


In [57]:
df

,timestamp,TS_id,site_id,system.SystemState,system.BMS_EMS_CommStatus,system.SafetyLoopStatus,system.SystemStatusFlags,system.CapacityChargekW,system.CapacityDischargekW,system.CapacitykVAR,...,pcs.PCSStatus_All_PCS,pcs.PF_Command_All_PCS,pcs.Actual_PF_All_PCS,pcs.P_Command_All_PCS,pcs.P_CommandAck_All_PCS,pcs.Q_Command_All_PCS,pcs.Q_CommandAck_All_PCS,pcs.RealPower_All_PCS,pcs.ReactivePower_All_PCS,pcs.ApparentPower_All_PCS
0,2023-10-19 11:00:05.6 PDT,1,relavant_path1,4,0,0,3423076352,3600,-3600,2520,...,1073741824,0,0,0,450,0,0,450,0,450
1,2023-10-19 11:00:05.6 PDT,2,relavant_path1,4,0,0,3423076352,3600,-3600,2520,...,1073741824,0,0,0,450,0,0,450,0,450
2,2023-10-19 11:00:05.6 PDT,3,relavant_path1,4,0,0,3423076352,3600,-3600,2520,...,1073741824,0,0,0,450,0,0,450,0,450
3,2023-10-19 11:00:05.6 PDT,4,relavant_path1,4,0,0,3423076352,3600,-3600,2520,...,1073741824,0,0,0,450,0,0,450,0,450
4,2023-10-19 11:00:05.6 PDT,5,relavant_path1,4,0,0,3423076352,3600,-3600,2520,...,1073741824,0,0,0,450,0,0,450,0,450
5,2023-10-19 11:00:05.6 PDT,6,relavant_path1,4,0,0,3423076352,3600,-3600,2520,...,1073741824,0,0,0,450,0,0,450,0,450
6,2023-10-19 11:00:05.6 PDT,7,relavant_path1,4,0,0,3423076352,3600,-3600,2520,...,1073741824,0,0,0,450,0,0,450,0,450
7,2023-10-19 11:00:05.6 PDT,8,relavant_path1,4,0,0,3423076352,3600,-3600,2520,...,1073741824,0,0,0,450,0,0,450,0,450


In [59]:
df.to_csv("/home/lohith/Downloads/relative_path1/preprocess_the data.csv")

In [58]:
type(input_data)

dict

In [29]:
import pandas as pd
import re

# Input data
input_data = {
	"timestamp":	"2023-10-19 11:00:05.6 PDT",
	"TS[1].system.SystemState":	4,
	"TS[1].system.BMS_EMS_CommStatus":	0,
	"TS[1].system.SafetyLoopStatus":	0,
	"TS[1].system.SystemStatusFlags":	3423076352,
	"TS[1].system.CapacityChargekW":	3600,
	"TS[1].system.CapacityDischargekW":	-3600,
	"TS[1].system.CapacitykVAR":	2520,
	"TS[1].system.SysVoltageDC":	1453,
	"TS[1].system.SysCurrentDC":	32,
	"TS[1].bms.BMSState":	0,
	"TS[1].bms.EventLog_LastEventNum":	0,
	"TS[1].pcs.PCS_MaxCapacitykW":	3600,
	"TS[1].pcs.PCS_MaxCapacitykVAR":	2520,
	"TS[1].pcs.PCSStatus_All_PCS":	1073741824,
	"TS[1].pcs.PF_Command_All_PCS":	0,
	"TS[1].pcs.Actual_PF_All_PCS":	0,
	"TS[1].pcs.P_Command_All_PCS":	0,
	"TS[1].pcs.P_CommandAck_All_PCS":	450,
	"TS[1].pcs.Q_Command_All_PCS":	0,
	"TS[1].pcs.Q_CommandAck_All_PCS":	0,
	"TS[1].pcs.RealPower_All_PCS":	450,
	"TS[1].pcs.ReactivePower_All_PCS":	0,
	"TS[1].pcs.ApparentPower_All_PCS":	450,
	"TS[2].system.SystemState":	4,
	"TS[2].system.BMS_EMS_CommStatus":	0,
	"TS[2].system.SafetyLoopStatus":	0,
	"TS[2].system.SystemStatusFlags":	3423076352,
	"TS[2].system.CapacityChargekW":	3600,
	"TS[2].system.CapacityDischargekW":	-3600,
	"TS[2].system.CapacitykVAR":	2520,
	"TS[2].system.SysVoltageDC":	1453,
	"TS[2].system.SysCurrentDC":	32,
	"TS[2].bms.BMSState":	0,
	"TS[2].bms.EventLog_LastEventNum":	0,
	"TS[2].pcs.PCS_MaxCapacitykW":	3600,
	"TS[2].pcs.PCS_MaxCapacitykVAR":	2520,
	"TS[2].pcs.PCSStatus_All_PCS":	1073741824,
	"TS[2].pcs.PF_Command_All_PCS":	0,
	"TS[2].pcs.Actual_PF_All_PCS":	0,
	"TS[2].pcs.P_Command_All_PCS":	0,
	"TS[2].pcs.P_CommandAck_All_PCS":	450,
	"TS[2].pcs.Q_Command_All_PCS":	0,
	"TS[2].pcs.Q_CommandAck_All_PCS":	0,
	"TS[2].pcs.RealPower_All_PCS":	450,
	"TS[2].pcs.ReactivePower_All_PCS":	0,
	"TS[2].pcs.ApparentPower_All_PCS":	450,
	"TS[3].system.SystemState":	4,
	"TS[3].system.BMS_EMS_CommStatus":	0,
	"TS[3].system.SafetyLoopStatus":	0,
	"TS[3].system.SystemStatusFlags":	3423076352,
	"TS[3].system.CapacityChargekW":	3600,
	"TS[3].system.CapacityDischargekW":	-3600,
	"TS[3].system.CapacitykVAR":	2520,
	"TS[3].system.SysVoltageDC":	1453,
	"TS[3].system.SysCurrentDC":	32,
	"TS[3].bms.BMSState":	0,
	"TS[3].bms.EventLog_LastEventNum":	0,
	"TS[3].pcs.PCS_MaxCapacitykW":	3600,
	"TS[3].pcs.PCS_MaxCapacitykVAR":	2520,
	"TS[3].pcs.PCSStatus_All_PCS":	1073741824,
	"TS[3].pcs.PF_Command_All_PCS":	0,
	"TS[3].pcs.Actual_PF_All_PCS":	0,
	"TS[3].pcs.P_Command_All_PCS":	0,
	"TS[3].pcs.P_CommandAck_All_PCS":	450,
	"TS[3].pcs.Q_Command_All_PCS":	0,
	"TS[3].pcs.Q_CommandAck_All_PCS":	0,
	"TS[3].pcs.RealPower_All_PCS":	450,
	"TS[3].pcs.ReactivePower_All_PCS":	0,
	"TS[3].pcs.ApparentPower_All_PCS":	450,
	"TS[4].system.SystemState":	4,
	"TS[4].system.BMS_EMS_CommStatus":	0,
	"TS[4].system.SafetyLoopStatus":	0,
	"TS[4].system.SystemStatusFlags":	3423076352,
	"TS[4].system.CapacityChargekW":	3600,
	"TS[4].system.CapacityDischargekW":	-3600,
	"TS[4].system.CapacitykVAR":	2520,
	"TS[4].system.SysVoltageDC":	1453,
	"TS[4].system.SysCurrentDC":	32,
	"TS[4].bms.BMSState":	0,
	"TS[4].bms.EventLog_LastEventNum":	0,
	"TS[4].pcs.PCS_MaxCapacitykW":	3600,
	"TS[4].pcs.PCS_MaxCapacitykVAR":	2520,
	"TS[4].pcs.PCSStatus_All_PCS":	1073741824,
	"TS[4].pcs.PF_Command_All_PCS":	0,
	"TS[4].pcs.Actual_PF_All_PCS":	0,
	"TS[4].pcs.P_Command_All_PCS":	0,
	"TS[4].pcs.P_CommandAck_All_PCS":	450,
	"TS[4].pcs.Q_Command_All_PCS":	0,
	"TS[4].pcs.Q_CommandAck_All_PCS":	0,
	"TS[4].pcs.RealPower_All_PCS":	450,
	"TS[4].pcs.ReactivePower_All_PCS":	0,
	"TS[4].pcs.ApparentPower_All_PCS":	450,
	"TS[5].system.SystemState":	4,
	"TS[5].system.BMS_EMS_CommStatus":	0,
	"TS[5].system.SafetyLoopStatus":	0,
	"TS[5].system.SystemStatusFlags":	3423076352,
	"TS[5].system.CapacityChargekW":	3600,
	"TS[5].system.CapacityDischargekW":	-3600,
	"TS[5].system.CapacitykVAR":	2520,
	"TS[5].system.SysVoltageDC":	1453,
	"TS[5].system.SysCurrentDC":	32,
	"TS[5].bms.BMSState":	0,
	"TS[5].bms.EventLog_LastEventNum":	0,
	"TS[5].pcs.PCS_MaxCapacitykW":	3600,
	"TS[5].pcs.PCS_MaxCapacitykVAR":	2520,
	"TS[5].pcs.PCSStatus_All_PCS":	1073741824,
	"TS[5].pcs.PF_Command_All_PCS":	0,
	"TS[5].pcs.Actual_PF_All_PCS":	0,
	"TS[5].pcs.P_Command_All_PCS":	0,
	"TS[5].pcs.P_CommandAck_All_PCS":	450,
	"TS[5].pcs.Q_Command_All_PCS":	0,
	"TS[5].pcs.Q_CommandAck_All_PCS":	0,
	"TS[5].pcs.RealPower_All_PCS":	450,
	"TS[5].pcs.ReactivePower_All_PCS":	0,
	"TS[5].pcs.ApparentPower_All_PCS":	450,
	"TS[6].system.SystemState":	4,
	"TS[6].system.BMS_EMS_CommStatus":	0,
	"TS[6].system.SafetyLoopStatus":	0,
	"TS[6].system.SystemStatusFlags":	3423076352,
	"TS[6].system.CapacityChargekW":	3600,
	"TS[6].system.CapacityDischargekW":	-3600,
	"TS[6].system.CapacitykVAR":	2520,
	"TS[6].system.SysVoltageDC":	1453,
	"TS[6].system.SysCurrentDC":	32,
	"TS[6].bms.BMSState":	0,
	"TS[6].bms.EventLog_LastEventNum":	0,
	"TS[6].pcs.PCS_MaxCapacitykW":	3600,
	"TS[6].pcs.PCS_MaxCapacitykVAR":	2520,
	"TS[6].pcs.PCSStatus_All_PCS":	1073741824,
	"TS[6].pcs.PF_Command_All_PCS":	0,
	"TS[6].pcs.Actual_PF_All_PCS":	0,
	"TS[6].pcs.P_Command_All_PCS":	0,
	"TS[6].pcs.P_CommandAck_All_PCS":	450,
	"TS[6].pcs.Q_Command_All_PCS":	0,
	"TS[6].pcs.Q_CommandAck_All_PCS":	0,
	"TS[6].pcs.RealPower_All_PCS":	450,
	"TS[6].pcs.ReactivePower_All_PCS":	0,
	"TS[6].pcs.ApparentPower_All_PCS":	450,
	"TS[7].system.SystemState":	4,
	"TS[7].system.BMS_EMS_CommStatus":	0,
	"TS[7].system.SafetyLoopStatus":	0,
	"TS[7].system.SystemStatusFlags":	3423076352,
	"TS[7].system.CapacityChargekW":	3600,
	"TS[7].system.CapacityDischargekW":	-3600,
	"TS[7].system.CapacitykVAR":	2520,
	"TS[7].system.SysVoltageDC":	1453,
	"TS[7].system.SysCurrentDC":	32,
	"TS[7].bms.BMSState":	0,
	"TS[7].bms.EventLog_LastEventNum":	0,
	"TS[7].pcs.PCS_MaxCapacitykW":	3600,
	"TS[7].pcs.PCS_MaxCapacitykVAR":	2520,
	"TS[7].pcs.PCSStatus_All_PCS":	1073741824,
	"TS[7].pcs.PF_Command_All_PCS":	0,
	"TS[7].pcs.Actual_PF_All_PCS":	0,
	"TS[7].pcs.P_Command_All_PCS":	0,
	"TS[7].pcs.P_CommandAck_All_PCS":	450,
	"TS[7].pcs.Q_Command_All_PCS":	0,
	"TS[7].pcs.Q_CommandAck_All_PCS":	0,
	"TS[7].pcs.RealPower_All_PCS":	450,
	"TS[7].pcs.ReactivePower_All_PCS":	0,
	"TS[7].pcs.ApparentPower_All_PCS":	450,
	"TS[8].system.SystemState":	4,
	"TS[8].system.BMS_EMS_CommStatus":	0,
	"TS[8].system.SafetyLoopStatus":	0,
	"TS[8].system.SystemStatusFlags":	3423076352,
	"TS[8].system.CapacityChargekW":	3600,
	"TS[8].system.CapacityDischargekW":	-3600,
	"TS[8].system.CapacitykVAR":	2520,
	"TS[8].system.SysVoltageDC":	1453,
	"TS[8].system.SysCurrentDC":	32,
	"TS[8].bms.BMSState":	0,
	"TS[8].bms.EventLog_LastEventNum":	0,
	"TS[8].pcs.PCS_MaxCapacitykW":	3600,
	"TS[8].pcs.PCS_MaxCapacitykVAR":	2520,
	"TS[8].pcs.PCSStatus_All_PCS":	1073741824,
	"TS[8].pcs.PF_Command_All_PCS":	0,
	"TS[8].pcs.Actual_PF_All_PCS":	0,
	"TS[8].pcs.P_Command_All_PCS":	0,
	"TS[8].pcs.P_CommandAck_All_PCS":	450,
	"TS[8].pcs.Q_Command_All_PCS":	0,
	"TS[8].pcs.Q_CommandAck_All_PCS":	0,
	"TS[8].pcs.RealPower_All_PCS":	450,
	"TS[8].pcs.ReactivePower_All_PCS":	0,
	"TS[8].pcs.ApparentPower_All_PCS":	450
}

# Function to extract information from the nested structure
def extract_info(data):
    # Initialize a list to store extracted values
    extracted_data_list = []
    
    # Extract data for each TS entry
    for key, value in data.items():
        if key != "timestamp":
            match = re.match(r"TS\[(\d+)\]\.(\w+)\.(\w+)", key)
            if match:
                ts_id, category, attribute = match.groups()
                extracted_data = {
                    "timestamp": data.get("timestamp", ""),
                    "TS_id": int(ts_id),
                    "site_id": "relative_path1",
                    f"{category}.{attribute}": value
                }
                extracted_data_list.append(extracted_data)

    return extracted_data_list

# Extract data from the input
extracted_data_list = extract_info(input_data)

# Create a DataFrame from the extracted data list
df = pd.DataFrame(extracted_data_list)

# Reorder the columns
expected_order = ["TS_id", "timestamp", "site_id"]
columns_to_flatten = ["system", "bms", "pcs"]
for column in columns_to_flatten:
    prefix = f"{column}."
    filtered_columns = [col for col in df.columns if col.startswith(prefix)]
    expected_order.extend(filtered_columns)

df = df[expected_order]

# Display the resulting DataFrame
print(df)


     TS_id                  timestamp         site_id  system.SystemState  \
0        1  2023-10-19 11:00:05.6 PDT  relative_path1                 4.0   
1        1  2023-10-19 11:00:05.6 PDT  relative_path1                 NaN   
2        1  2023-10-19 11:00:05.6 PDT  relative_path1                 NaN   
3        1  2023-10-19 11:00:05.6 PDT  relative_path1                 NaN   
4        1  2023-10-19 11:00:05.6 PDT  relative_path1                 NaN   
..     ...                        ...             ...                 ...   
179      8  2023-10-19 11:00:05.6 PDT  relative_path1                 NaN   
180      8  2023-10-19 11:00:05.6 PDT  relative_path1                 NaN   
181      8  2023-10-19 11:00:05.6 PDT  relative_path1                 NaN   
182      8  2023-10-19 11:00:05.6 PDT  relative_path1                 NaN   
183      8  2023-10-19 11:00:05.6 PDT  relative_path1                 NaN   

     system.BMS_EMS_CommStatus  system.SafetyLoopStatus  \
0               